# 🤖 Add AI Documentation Processor: Multi-Agent Content Analysis & Code Generation System

This PR adds a comprehensive AI-powered documentation processing system using PraisonAI and OpenAI gpt-5-nano. Features 3 specialized agents for content analysis, code generation, and example creation, perfect for automated documentation processing and code generation workflows.

**Key Features:** Multi-agent document analysis, automated code generation, usage example creation, content validation, and scalable processing workflows for various document formats.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DhivyaBharathy-web/PraisonAI/blob/main/examples/cookbooks/ai_documentation_processor.ipynb)


In [5]:
# @title **📦 Install Dependencies**

# Install required packages
!pip install praisonaiagents openai python-dotenv fastapi uvicorn -q

# Basic imports
import os
import json
import re  # FIXED: Added missing import
import base64
import mimetypes
import logging
from typing import Dict, List, Optional, Any, Tuple
from datetime import datetime
import getpass

print("📦 Dependencies installed successfully!")
print("📝 AI Documentation Processor ready!")
print("🤖 Using PraisonAI Multi-Agent Framework")

📦 Dependencies installed successfully!
📝 AI Documentation Processor ready!
🤖 Using PraisonAI Multi-Agent Framework


In [6]:
# @title ** Set OpenAI API Key**

# Set up API key
print("🔑 OpenAI API Key Setup")
print("Enter your OpenAI API key:")

api_key = getpass.getpass("OpenAI API Key: ")

if not api_key or api_key.strip() == "":
    print("⚠️  No API key provided. Using demo mode with mock responses.")
    api_key = "demo-key"

# Set environment variable
os.environ["OPENAI_API_KEY"] = api_key

print("✅ API key configured!")
print("🚀 Ready to process documentation!")

🔑 OpenAI API Key Setup
Enter your OpenAI API key:
OpenAI API Key: ··········
✅ API key configured!
🚀 Ready to process documentation!


In [7]:
# @title **️ Custom Processing Tools (FIXED)**

class DocumentationTools:
    """Custom tools for documentation processing and validation"""

    @staticmethod
    def clean_code(code: str) -> str:
        """Clean and format code"""
        # Remove markdown code blocks if present
        if "```typescript" in code:
            code = code.split("```typescript")[1]
        if "```python" in code:
            code = code.split("```python")[1]
        if "```" in code:
            code = code.split("```")[0]

        # Clean up extra whitespace
        code = code.strip()
        return code

    @staticmethod
    def extract_class_name(code: str) -> str:
        """Extract class name from code"""
        # Look for class definitions
        pattern = r'class\s+(\w+)\s*{'
        match = re.search(pattern, code)
        if match:
            return match.group(1)
        return ""

    @staticmethod
    def validate_code_syntax(code: str) -> Tuple[bool, str]:
        """Basic code syntax validation"""
        try:
            # Check for basic patterns
            if "interface" in code or "class" in code or "async" in code or "function" in code:
                return True, "Code patterns found"
            else:
                return False, "No code patterns detected"
        except Exception as e:
            return False, f"Validation error: {str(e)}"

    @staticmethod
    def check_api_patterns(code: str) -> Tuple[bool, str]:
        """Check if code contains API-related patterns"""
        api_patterns = ['fetch', 'Promise', 'async', 'await', 'interface', 'class']
        found_patterns = []

        for pattern in api_patterns:
            if pattern in code:
                found_patterns.append(pattern)

        if found_patterns:
            return True, f"Found patterns: {', '.join(found_patterns)}"
        else:
            return False, "No relevant patterns found"

    @staticmethod
    def generate_code_summary(code: str) -> Dict[str, Any]:
        """Generate a summary of the code"""
        class_name = DocumentationTools.extract_class_name(code)
        syntax_valid, syntax_msg = DocumentationTools.validate_code_syntax(code)
        patterns_ok, patterns_msg = DocumentationTools.check_api_patterns(code)

        return {
            "class_name": class_name,
            "syntax_valid": syntax_valid,
            "syntax_message": syntax_msg,
            "patterns_ok": patterns_ok,
            "patterns_message": patterns_msg,
            "code_length": len(code),
            "lines": len(code.split('\n'))
        }

# Initialize tools
doc_tools = DocumentationTools()
print("✅ Custom documentation tools initialized!")

✅ Custom documentation tools initialized!


In [8]:
# @title **🚀 Execute Documentation Processing**

# Import PraisonAI after setting API key
from praisonaiagents import Agent, Task, PraisonAIAgents

def process_documentation(content: str, output_type: str = "code"):
    """Process documentation and generate specialized outputs using PraisonAI"""
    print(f"📝 Processing documentation for: {output_type}")
    print("=" * 60)

    # Create specialized agents
    analysis_agent = Agent(
        name="Content Analysis Agent",
        role="Documentation Analyst",
        goal="Extract structured information from documentation",
        backstory="Expert in analyzing documentation to extract key information, patterns, and requirements",
        verbose=True,
        allow_delegation=False,
        tools=[]
    )

    generation_agent = Agent(
        name="Code Generation Agent",
        role="Code Developer",
        goal="Generate specialized code based on content analysis",
        backstory="Expert developer specializing in creating code based on documentation and requirements",
        verbose=True,
        allow_delegation=False,
        tools=[]
    )

    example_agent = Agent(
        name="Example Creation Agent",
        role="Documentation Specialist",
        goal="Create comprehensive examples and usage patterns",
        backstory="Expert in creating clear, practical examples that demonstrate functionality",
        verbose=True,
        allow_delegation=False,
        tools=[]
    )

    # Step 1: Analyze Content
    print("🔍 Step 1: Analyzing documentation content...")

    analysis_task = Task(
        description=f"""
        Analyze this documentation content and extract key information:

        Content:
        {content}

        Extract:
        1. Main topics and concepts
        2. Key requirements or specifications
        3. Data structures or patterns
        4. Any technical details or constraints

        Return a structured summary of the content.
        """,
        agent=analysis_agent
    )

    # Execute content analysis
    analysis_system = PraisonAIAgents(
        agents=[analysis_agent],
        tasks=[analysis_task],
        verbose=True,
        process="sequential"
    )

    content_analysis = analysis_system.start()
    print(f"✅ Content analysis completed: {content_analysis[:200]}...")

    # Step 2: Generate Code
    print("�� Step 2: Generating specialized code...")

    generation_task = Task(
        description=f"""
        Generate specialized code based on this content analysis:

        Content Analysis: {content_analysis}
        Original Content: {content}
        Output Type: {output_type}

        Create appropriate code that:
        1. Implements the requirements from the analysis
        2. Uses proper syntax and structure
        3. Includes necessary imports and dependencies
        4. Has clear comments and documentation
        5. Is ready to use or extend

        Return ONLY the code, no explanations.
        """,
        agent=generation_agent
    )

    # Execute code generation
    generation_system = PraisonAIAgents(
        agents=[generation_agent],
        tasks=[generation_task],
        verbose=True,
        process="sequential"
    )

    generated_code = generation_system.start()
    generated_code = doc_tools.clean_code(generated_code)
    print(f"✅ Code generated: {generated_code[:200]}...")

    # Step 3: Generate Examples
    print("�� Step 3: Generating usage examples...")

    example_task = Task(
        description=f"""
        Create comprehensive usage examples for this generated code:

        Generated Code: {generated_code}
        Content Analysis: {content_analysis}

        Generate examples that:
        1. Demonstrate how to use the generated code
        2. Show different use cases and scenarios
        3. Include proper error handling
        4. Are clear and well-documented
        5. Can be easily understood and followed

        Return ONLY the example code, no explanations.
        """,
        agent=example_agent
    )

    # Execute example generation
    example_system = PraisonAIAgents(
        agents=[example_agent],
        tasks=[example_task],
        verbose=True,
        process="sequential"
    )

    usage_examples = example_system.start()
    usage_examples = doc_tools.clean_code(usage_examples)
    print(f"✅ Usage examples generated: {usage_examples[:200]}...")

    # Step 4: Validate generated code
    print("✅ Step 4: Validating generated code...")
    code_summary = doc_tools.generate_code_summary(generated_code)

    print(f"📊 Code Summary:")
    print(f"   - Class/Function: {code_summary['class_name']}")
    print(f"   - Syntax Valid: {code_summary['syntax_valid']}")
    print(f"   - Patterns Found: {code_summary['patterns_ok']}")
    print(f"   - Code Length: {code_summary['code_length']} characters")
    print(f"   - Lines: {code_summary['lines']}")

    return {
        "generated_code": generated_code,
        "usage_examples": usage_examples,
        "content_analysis": content_analysis,
        "code_summary": code_summary,
        "output_type": output_type
    }

# Demo execution
print("📝 AI Documentation Processor Ready!")
print("=" * 60)

# Example documentation
demo_content = """
# User Management System

## Overview
This system manages user accounts with basic CRUD operations.

## User Model
- id: unique identifier
- name: user's full name
- email: user's email address
- created_at: timestamp

## API Endpoints
- GET /users - List all users
- POST /users - Create new user
- GET /users/{id} - Get user by ID
- PUT /users/{id} - Update user
- DELETE /users/{id} - Delete user
"""

print("🚀 Demo: Processing documentation...")
result = process_documentation(demo_content, "TypeScript SDK")

print("\n�� Generated Code:")
print("=" * 40)
print(result['generated_code'])
print("=" * 40)

print("\n�� Usage Examples:")
print("=" * 40)
print(result['usage_examples'])
print("=" * 40)

print(f"\n✅ Documentation processing complete!")
print(f"📊 Output type: {result['output_type']}")
print(f"📈 Code quality: {result['code_summary']['lines']} lines, {result['code_summary']['code_length']} characters")

📝 AI Documentation Processor Ready!
🚀 Demo: Processing documentation...
📝 Processing documentation for: TypeScript SDK
🔍 Step 1: Analyzing documentation content...


╭─ Agent Info ────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                                                                                                 │
│  👤 Agent: Content Analysis Agent                                                                               │
│  Role: Documentation Analyst                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Response generated in 3.6s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You need to do the following task: Analyze this documentation content and extract key information:              │
│                                                                                                                 │
│                                                                                                                 │
│      Content:                                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃                                           User Management System                                            ┃ │
│ ┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛ │
│                                                                                                                 │
│                                                                                                                 │
│                                                    Overview                                                     │
│                                                                                                                 │
│ This system manages user accounts with basic CRUD operations.                                                   │
│                                                                                                                 │
│                                                                                                                 │
│                                                   User Model                                                    │
│                                                                                                                 │
│  • id: unique identifier                                                                                        │
│  • name: user's full name                                                                                       │
│  • email: user's email address                                                                                  │
│  • created_at: timestamp                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
│                                                  API Endpoints                                                  │
│                                                                                                                 │
│  • GET /users - List all users                                                                                  │
│  • POST /users - Create new user                                                                                │
│  • GET /users/{id} - Get user by ID                                                                             │
│  • PUT /users/{id} - Update user                                                                                │
│  • DELETE /users/{id} - Delete user                                                                             │
│                                                                                                                 │
│       Extract:                                                                                                  │
│       1. Main topics and concepts                     

...

Output()

Response generated in 5.0s

╭───────────────────────────────────────────────────── Task ──────────────────────────────────────────────────────╮
│ You need to do the following task: Create comprehensive usage examples for this generated code:                 │
│                                                                                                                 │
│                                                                                                                 │
│      Generated Code: // Import necessary modules                                                                │
│                                                                                                                 │
│                                                                                                                 │
│ import express, { Request, Response } from 'express'; import { v4 as uuidv4 } from 'uuid';                      │
│                                                                                                                 │
│ // Define the User model interface User { id: string; name: string; email: string; created_at: Date; }          │
│                                                                                                                 │
│ // In-memory user storage const users: User[] = [];                                                             │
│                                                                                                                 │
│ // Initialize Express app const app = express(); app.use(express.json());                                       │
│                                                                                                                 │
│ // API Endpoints                                                                                                │
│                                                                                                                 │
│ // GET /users - List all users app.get('/users', (req: Request, res: Response) => { res.json(users); });        │
│                                                                                                                 │
│ // POST /users - Create new user app.post('/users', (req: Request, res: Response) => { const { name, email } =  │
│ req.body; const newUser: User = { id: uuidv4(), name, email, created_at: new Date(), }; users.push(newUser);    │
│ res.status(201).json(newUser); });                                                                              │
│                                                                                                                 │
│ // GET /users/{id} - Get user by ID app.get('/users/:id', (req: Request, res: Response) => { const user =       │
│ users.find(u => u.id === req.params.id); if (user) { res.json(user); } else { res.status(404).send('User not    │
│ found'); } });                                                                                                  │
│                                                                                                                 │
│ // PUT /users/{id} - Update user app.put('/users/:id', (req: Request, res: Response) => { const user =          │
│ users.find(u => u.id === req.params.id); if (user) { const { name, email } = req.body; user.name = name ||      │
│ user.name; user.email = email || user.email; res.json(user); } else { res.status(404).send('User not found'); } │
│ });                                                                                                             │
│                                                                                                                 │
│ // DELETE /users/{id} - Delete user app.delete('/users/:id', (req: Request, res: Response) => { const userIndex │
│ = users.findIndex(u => u.id === req.params.id); if (userIndex !== -1) { users.splice(userIndex, 1);             │
│ res.status(204).send(); } else { res.status(404).send(

╭─────────────────────────────────────────────────── Response ────────────────────────────────────────────────────╮
│                                                                                                                 │
│  // Example Usage of the User Management System                                                                 │
│                                                                                                                 │
│  // Import necessary modules                                                                                    │
│  import express, { Request, Response } from 'express';                                                          │
│  import { v4 as uuidv4 } from 'uuid';                                                                           │
│                                                                                                                 │
│  // Define the User model                                                                                       │
│  interface User {                                                                                               │
│    id: string;                                                                                                  │
│    name: string;                                                                                                │
│    email: string;                                                                                               │
│    created_at: Date;                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│  // In-memory user storage                                                                                      │
│  const users: User[] = [];                                                                                      │
│                                                                                                                 │
│  // Initialize Express app                                                                                      │
│  const app = express();                                                                                         │
│  app.use(express.json());                                                                                       │
│                                                                                                                 │
│  // API Endpoints                                                                                               │
│                                                                                                                 │
│  // GET /users - List all users                                                                                 │
│  app.get('/users', (req: Request, res: Response) => {                                                           │
│    res.json(users);                                                                                             │
│  });                                                                                                            │
│                                                                                                                 │
│  // POST /users - Create new user                                                                               │
│  app.post('/users', (req: Request, res: Response) => {                                                          │
│    const { name, email } = req.body;                                                                            │
│    if (!name || !email) {                                                                                       │
│      return res.status(400).send('Name and email are r

✅ Usage examples generated: ...
✅ Step 4: Validating generated code...
📊 Code Summary:
   - Class/Function: 
   - Syntax Valid: True
   - Patterns Found: True
   - Code Length: 1821 characters
   - Lines: 78

�� Generated Code:
// Import necessary modules
import express, { Request, Response } from 'express';
import { v4 as uuidv4 } from 'uuid';

// Define the User model
interface User {
  id: string;
  name: string;
  email: string;
  created_at: Date;
}

// In-memory user storage
const users: User[] = [];

// Initialize Express app
const app = express();
app.use(express.json());

// API Endpoints

// GET /users - List all users
app.get('/users', (req: Request, res: Response) => {
  res.json(users);
});

// POST /users - Create new user
app.post('/users', (req: Request, res: Response) => {
  const { name, email } = req.body;
  const newUser: User = {
    id: uuidv4(),
    name,
    email,
    created_at: new Date(),
  };
  users.push(newUser);
  res.status(201).json(newUser);
});

// G